In [9]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# PLAN DE TRABAJO: CONCATENACIÓN DE DATOS NOROESTE
# =================================================
# 
# OBJETIVO: Concatenar datos de NOROESTE, NOROESTE2 y NOROESTE3 de todos los archivos
# 
# ESTRUCTURAS IDENTIFICADAS:
# 1. 2020-2021, 2022-2023: Hojas separadas (NOROESTE, NOROESTE2, NOROESTE3)
# 2. 2023-2024: Una hoja con columnas agrupadas (NOROESTE, NOROESTE 2, NOROESTE 3)
# 3. 2024: Hojas separadas con nombres diferentes (NE, NE2, NE3)
#
# ESTRATEGIA:
# - Función específica para cada tipo de estructura
# - Estandarización de nombres de columnas
# - Concatenación temporal de todas las estaciones
# - Control de calidad de datos

print("INICIANDO PLAN DE CONCATENACIÓN DE DATOS NOROESTE")
print("=" * 60)

INICIANDO PLAN DE CONCATENACIÓN DE DATOS NOROESTE


In [10]:
# FUNCIÓN 1: Leer datos de archivos con hojas separadas (2020-2021, 2022-2023)
def read_noroeste_separate_sheets(file_path, year_range):
    """
    Lee datos de NOROESTE, NOROESTE2 y NOROESTE3 de archivos con hojas separadas
    """
    print(f"Leyendo datos {year_range} (hojas separadas)...")
    
    noroeste_data = {}
    stations = ['NOROESTE', 'NOROESTE2', 'NOROESTE3']
    
    try:
        for station in stations:
            df = pd.read_excel(file_path, sheet_name=station)
            
            # Estandarizar nombre de columna de fecha
            if 'date' in df.columns:
                df = df.rename(columns={'date': 'datetime'})
            
            # Añadir información de estación y período
            df['station'] = station
            df['year_range'] = year_range
            df['source_file'] = os.path.basename(file_path)
            
            noroeste_data[station] = df
            print(f"  [OK] {station}: {len(df)} registros")
            
    except Exception as e:
        print(f"  [ERROR] Error leyendo {year_range}: {e}")
        return None
    
    return noroeste_data

# Test con archivo 2020-2021
file_2020_2021 = 'Bases_Datos/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx'
data_2020_2021 = read_noroeste_separate_sheets(file_2020_2021, '2020-2021')

Leyendo datos 2020-2021 (hojas separadas)...
  [OK] NOROESTE: 17537 registros
  [OK] NOROESTE: 17537 registros
  [OK] NOROESTE2: 17535 registros
  [OK] NOROESTE3: 0 registros
  [OK] NOROESTE2: 17535 registros
  [OK] NOROESTE3: 0 registros


In [11]:
# FUNCIÓN 2: Leer datos de archivo con columnas agrupadas (2023-2024)
def read_noroeste_grouped_columns(file_path, year_range):
    """
    Lee datos de NOROESTE del archivo 2023-2024 donde están en columnas agrupadas
    """
    print(f"Leyendo datos {year_range} (columnas agrupadas)...")
    
    try:
        # Leer el archivo completo
        df_full = pd.read_excel(file_path, sheet_name='Param_horarios_Estaciones')
        
        noroeste_data = {}
        
        # Identificar grupos de columnas para cada estación NOROESTE
        station_groups = {
            'NOROESTE': [col for col in df_full.columns if col.startswith('NOROESTE') and not col.startswith('NOROESTE ')],
            'NOROESTE2': [col for col in df_full.columns if col.startswith('NOROESTE 2')],
            'NOROESTE3': [col for col in df_full.columns if col.startswith('NOROESTE 3')]
        }
        
        for station, cols in station_groups.items():
            if cols:
                # Extraer datos de las columnas correspondientes
                station_df = df_full[['Unnamed: 0'] + cols].copy()
                
                # La primera fila contiene los nombres de parámetros
                # La segunda fila contiene las unidades
                # Los datos empiezan desde la tercera fila
                param_names = station_df.iloc[0, 1:].values  # Parámetros
                units = station_df.iloc[1, 1:].values        # Unidades
                
                # Crear nuevos nombres de columnas combinando parámetro y unidad
                new_columns = ['datetime']
                for param, unit in zip(param_names, units):
                    if pd.notna(param) and pd.notna(unit):
                        new_columns.append(f"{param}")
                    elif pd.notna(param):
                        new_columns.append(f"{param}")
                    else:
                        new_columns.append("unknown")
                
                # Crear DataFrame con los datos (desde fila 2 en adelante)
                data_rows = station_df.iloc[2:].reset_index(drop=True)
                data_rows.columns = new_columns[:len(data_rows.columns)]
                
                # Añadir información de estación y período
                data_rows['station'] = station
                data_rows['year_range'] = year_range
                data_rows['source_file'] = os.path.basename(file_path)
                
                noroeste_data[station] = data_rows
                print(f"  [OK] {station}: {len(data_rows)} registros, {len(cols)} columnas")
            else:
                print(f"  [WARNING] {station}: No se encontraron columnas")
                
    except Exception as e:
        print(f"  [ERROR] Error leyendo {year_range}: {e}")
        return None
    
    return noroeste_data

# Test con archivo 2023-2024
file_2023_2024 = 'Bases_Datos/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx'
data_2023_2024 = read_noroeste_grouped_columns(file_2023_2024, '2023-2024')

Leyendo datos 2023-2024 (columnas agrupadas)...
  [OK] NOROESTE: 13870 registros, 15 columnas
  [OK] NOROESTE2: 13870 registros, 15 columnas
  [OK] NOROESTE3: 13870 registros, 15 columnas
  [OK] NOROESTE: 13870 registros, 15 columnas
  [OK] NOROESTE2: 13870 registros, 15 columnas
  [OK] NOROESTE3: 13870 registros, 15 columnas


In [12]:
# FUNCIÓN 3: Leer datos de archivo 2024 (hojas con nombres diferentes)
def read_noroeste_renamed_sheets(file_path, year_range):
    """
    Lee datos de NOROESTE del archivo 2024 donde las hojas tienen nombres NE, NE2, NE3
    """
    print(f"Leyendo datos {year_range} (hojas renombradas)...")
    
    noroeste_data = {}
    # Mapeo de nombres de hojas a nombres de estaciones
    sheet_mapping = {
        'NE': 'NOROESTE',
        'NE2': 'NOROESTE2', 
        'NE3': 'NOROESTE3'
    }
    
    try:
        for sheet_name, station_name in sheet_mapping.items():
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            # Estandarizar nombre de columna de fecha
            if 'Fecha y hora' in df.columns:
                df = df.rename(columns={'Fecha y hora': 'datetime'})
            
            # Limpiar nombres de columnas (remover unidades entre paréntesis para estandarizar)
            df.columns = [col.split(' (')[0] if '(' in col else col for col in df.columns]
            
            # Añadir información de estación y período
            df['station'] = station_name
            df['year_range'] = year_range
            df['source_file'] = os.path.basename(file_path)
            
            noroeste_data[station_name] = df
            print(f"  [OK] {station_name} (hoja {sheet_name}): {len(df)} registros")
            
    except Exception as e:
        print(f"  [ERROR] Error leyendo {year_range}: {e}")
        return None
    
    return noroeste_data

# Test con archivo 2024
file_2024 = 'Bases_Datos/DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx'
data_2024 = read_noroeste_renamed_sheets(file_2024, '2024')

Leyendo datos 2024 (hojas renombradas)...
  [OK] NOROESTE (hoja NE): 8784 registros
  [OK] NOROESTE (hoja NE): 8784 registros
  [OK] NOROESTE2 (hoja NE2): 8782 registros
  [OK] NOROESTE2 (hoja NE2): 8782 registros
  [OK] NOROESTE3 (hoja NE3): 8782 registros
  [OK] NOROESTE3 (hoja NE3): 8782 registros


In [13]:
# FUNCIÓN 4: Estandarizar columnas y concatenar datos
def standardize_and_concatenate(all_data_dict):
    """
    Estandariza nombres de columnas y concatena todos los datos en un DataFrame único
    """
    print("Estandarizando y concatenando datos...")
    
    all_dataframes = []
    
    # Mapeo de nombres de columnas comunes para estandarización
    column_mapping = {
        'CO': 'CO', 'co': 'CO',
        'NO': 'NO', 'no': 'NO', 
        'NO2': 'NO2', 'no2': 'NO2',
        'NOX': 'NOX', 'nox': 'NOX',
        'O3': 'O3', 'o3': 'O3',
        'PM10': 'PM10', 'pm10': 'PM10',
        'PM2.5': 'PM25', 'pm2.5': 'PM25', 'PM2': 'PM25',
        'PRS': 'PRS', 'prs': 'PRS',
        'RAINF': 'RAINF', 'rainf': 'RAINF',
        'RH': 'RH', 'rh': 'RH',
        'TEMP': 'TEMP', 'temp': 'TEMP',
        'WSR': 'WSR', 'wsr': 'WSR',
        'WDR': 'WDR', 'wdr': 'WDR'
    }
    
    for year_range, year_data in all_data_dict.items():
        if year_data is None:
            continue
            
        for station, df in year_data.items():
            if df is not None and not df.empty:
                # Crear copia para no modificar original
                df_clean = df.copy()
                
                # Estandarizar nombres de columnas
                df_clean.columns = [column_mapping.get(col, col) for col in df_clean.columns]
                
                # Asegurar que datetime sea datetime
                if 'datetime' in df_clean.columns:
                    df_clean['datetime'] = pd.to_datetime(df_clean['datetime'], errors='coerce')
                
                # Añadir a la lista
                all_dataframes.append(df_clean)
                print(f"  [OK] Añadido: {station} ({year_range}) - {len(df_clean)} registros")
    
    if all_dataframes:
        # Concatenar todos los DataFrames
        final_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
        
        # Ordenar por datetime
        final_df = final_df.sort_values('datetime').reset_index(drop=True)
        
        print(f"\nCONCATENACIÓN COMPLETADA:")
        print(f"   Total de registros: {len(final_df)}")
        print(f"   Rango de fechas: {final_df['datetime'].min()} a {final_df['datetime'].max()}")
        print(f"   Estaciones incluidas: {final_df['station'].unique()}")
        print(f"   Columnas disponibles: {len(final_df.columns)}")
        
        return final_df
    else:
        print("[ERROR] No se encontraron datos para concatenar")
        return None

# FUNCIÓN 5: Función principal para ejecutar todo el proceso
def process_all_noroeste_data():
    """
    Función principal que ejecuta todo el proceso de concatenación
    """
    print("INICIANDO PROCESO COMPLETO DE CONCATENACIÓN")
    print("=" * 60)
    
    all_data = {}
    
    # 1. Leer datos 2020-2021
    file_2020_2021 = 'Bases_Datos/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx'
    if os.path.exists(file_2020_2021):
        all_data['2020-2021'] = read_noroeste_separate_sheets(file_2020_2021, '2020-2021')
    
    # 2. Leer datos 2022-2023
    file_2022_2023 = 'Bases_Datos/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx'
    if os.path.exists(file_2022_2023):
        all_data['2022-2023'] = read_noroeste_separate_sheets(file_2022_2023, '2022-2023')
    
    # 3. Leer datos 2023-2024
    file_2023_2024 = 'Bases_Datos/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx'
    if os.path.exists(file_2023_2024):
        all_data['2023-2024'] = read_noroeste_grouped_columns(file_2023_2024, '2023-2024')
    
    # 4. Leer datos 2024
    file_2024 = 'Bases_Datos/DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx'
    if os.path.exists(file_2024):
        all_data['2024'] = read_noroeste_renamed_sheets(file_2024, '2024')
    
    # 5. Estandarizar y concatenar
    final_dataset = standardize_and_concatenate(all_data)
    
    return final_dataset, all_data

In [14]:
# EJECUTAR EL PROCESO COMPLETO
print("EJECUTANDO PROCESO DE CONCATENACIÓN COMPLETO")
print("=" * 60)

# Ejecutar proceso principal
noroeste_dataset, raw_data = process_all_noroeste_data()

# Mostrar resumen de resultados
if noroeste_dataset is not None:
    print("\nRESUMEN DETALLADO:")
    print("-" * 40)
    
    # Información por estación
    station_summary = noroeste_dataset.groupby('station').agg({
        'datetime': ['count', 'min', 'max'],
        'year_range': 'unique'
    })
    
    print("Registros por estación:")
    for station in noroeste_dataset['station'].unique():
        station_data = noroeste_dataset[noroeste_dataset['station'] == station]
        print(f"  {station}:")
        print(f"     Registros: {len(station_data)}")
        print(f"     Fechas: {station_data['datetime'].min()} a {station_data['datetime'].max()}")
        print(f"     Períodos: {', '.join(station_data['year_range'].unique())}")
        print()
    
    # Verificar columnas comunes
    print("ANÁLISIS DE COLUMNAS:")
    print("-" * 30)
    parameter_columns = [col for col in noroeste_dataset.columns 
                        if col not in ['datetime', 'station', 'year_range', 'source_file']]
    print(f"Parámetros disponibles: {parameter_columns}")
    
    # Verificar calidad de datos
    print("\nCALIDAD DE DATOS:")
    print("-" * 25)
    for param in parameter_columns[:5]:  # Mostrar primeros 5 parámetros
        if param in noroeste_dataset.columns:
            missing_pct = (noroeste_dataset[param].isna().sum() / len(noroeste_dataset)) * 100
            print(f"  {param}: {missing_pct:.1f}% datos faltantes")
else:
    print("[ERROR] No se pudo generar el dataset final")

EJECUTANDO PROCESO DE CONCATENACIÓN COMPLETO
INICIANDO PROCESO COMPLETO DE CONCATENACIÓN
Leyendo datos 2020-2021 (hojas separadas)...
  [OK] NOROESTE: 17537 registros
  [OK] NOROESTE: 17537 registros
  [OK] NOROESTE2: 17535 registros
  [OK] NOROESTE3: 0 registros
Leyendo datos 2022-2023 (hojas separadas)...
  [OK] NOROESTE2: 17535 registros
  [OK] NOROESTE3: 0 registros
Leyendo datos 2022-2023 (hojas separadas)...
  [OK] NOROESTE: 14255 registros
  [OK] NOROESTE: 14255 registros
  [OK] NOROESTE2: 14255 registros
  [OK] NOROESTE2: 14255 registros
  [OK] NOROESTE3: 6237 registros
Leyendo datos 2023-2024 (columnas agrupadas)...
  [OK] NOROESTE3: 6237 registros
Leyendo datos 2023-2024 (columnas agrupadas)...
  [OK] NOROESTE: 13870 registros, 15 columnas
  [OK] NOROESTE2: 13870 registros, 15 columnas
  [OK] NOROESTE3: 13870 registros, 15 columnas
Leyendo datos 2024 (hojas renombradas)...
  [OK] NOROESTE: 13870 registros, 15 columnas
  [OK] NOROESTE2: 13870 registros, 15 columnas
  [OK] NORO

In [15]:
# GUARDAR DATOS PROCESADOS
if noroeste_dataset is not None:
    print("GUARDANDO DATOS PROCESADOS...")
    
    # Guardar como CSV
    output_file = 'NOROESTE_CONCATENATED_DATA.csv'
    noroeste_dataset.to_csv(output_file, index=False)
    print(f"[OK] Datos guardados en: {output_file}")
    
    # Guardar resumen por estación
    summary_file = 'NOROESTE_SUMMARY.csv'
    station_summary = noroeste_dataset.groupby(['station', 'year_range']).agg({
        'datetime': ['count', 'min', 'max']
    }).reset_index()
    station_summary.to_csv(summary_file, index=False)
    print(f"[OK] Resumen guardado en: {summary_file}")
    
    print(f"\nPROCESO COMPLETADO EXITOSAMENTE!")
    print(f"Dataset final: {len(noroeste_dataset)} registros")
    print(f"Estaciones: {len(noroeste_dataset['station'].unique())}")
    print(f"Rango temporal: {noroeste_dataset['datetime'].min()} a {noroeste_dataset['datetime'].max()}")

# PRÓXIMOS PASOS RECOMENDADOS
print("\nPRÓXIMOS PASOS RECOMENDADOS:")
print("=" * 40)
print("1. Análisis exploratorio de datos (EDA)")
print("2. Limpieza adicional (outliers, valores inconsistentes)")
print("3. Visualización temporal por estación")
print("4. Análisis de correlaciones entre estaciones")
print("5. Identificación de patrones estacionales")
print("6. Detección de anomalías")
print("7. Preparación para análisis específicos")

print("\n" + "="*60)
print("CONCATENACIÓN DE DATOS NOROESTE FINALIZADA")
print("="*60)

GUARDANDO DATOS PROCESADOS...
[OK] Datos guardados en: NOROESTE_CONCATENATED_DATA.csv
[OK] Resumen guardado en: NOROESTE_SUMMARY.csv

PROCESO COMPLETADO EXITOSAMENTE!
Dataset final: 137777 registros
Estaciones: 3
Rango temporal: 2020-01-01 00:00:00 a 2024-12-31 23:00:00

PRÓXIMOS PASOS RECOMENDADOS:
1. Análisis exploratorio de datos (EDA)
2. Limpieza adicional (outliers, valores inconsistentes)
3. Visualización temporal por estación
4. Análisis de correlaciones entre estaciones
5. Identificación de patrones estacionales
6. Detección de anomalías
7. Preparación para análisis específicos

CONCATENACIÓN DE DATOS NOROESTE FINALIZADA
[OK] Datos guardados en: NOROESTE_CONCATENATED_DATA.csv
[OK] Resumen guardado en: NOROESTE_SUMMARY.csv

PROCESO COMPLETADO EXITOSAMENTE!
Dataset final: 137777 registros
Estaciones: 3
Rango temporal: 2020-01-01 00:00:00 a 2024-12-31 23:00:00

PRÓXIMOS PASOS RECOMENDADOS:
1. Análisis exploratorio de datos (EDA)
2. Limpieza adicional (outliers, valores inconsisten